# Project 2
Jason Zhongyun Zhang

In [1]:
import pandas as pd
import numpy as np
import quandl

# Data Importing, Cleaning, and Basic Manipulation

In [2]:
# to import the tickers from csv file
tickers = pd.read_csv('tickers.csv')
tickers = tickers.values.ravel()

In [1]:
# to write a function to import and concatenate the data for different stocks
def importdata (tickers):
    
    output = quandl.get(str('EOD/'+tickers[0]), start_date = '2000-01-01', end_date = '2019-01-10')[['Adj_Close', 'Volume']]
    output = pd.DataFrame(output)
    output['ticker'] = tickers[0]


    for ticker in tickers[1:]:
        try:
            data = quandl.get(str('EOD/'+ticker), start_date = '2000-01-01', end_date = '2019-01-10')[['Adj_Close', 'Volume']]
            data = pd.DataFrame(data)
            data['ticker'] = ticker
            output = pd.concat([output, data], axis=0)
        except KeyError:
            print(ticker)
            pass
        except RemoteDataError as exp:
            print(ticker)
            pass
        except NameError:
            print(ticker)
            pass
        
        #except NotFoundError:
            #print(ticker)
            #pass

    output.reset_index(inplace= True)
    output.rename(columns={output.columns[0]: "date"}, inplace = True)
    
    return output

In [14]:
df = importdata(tickers)

In [ ]:
#to check NA values from the imported data

#df.dropna().shape
#df.groupby(by="ticker").close.fillna(method='ffill', inplace=True)
#no NA values in initial, but Nans appear in later dataframes after manipulations

In [16]:
df = pd.DataFrame(df)
df.rename(columns={df.columns[1]: "Adj Close"}, inplace = True)
df.set_index(['ticker','date'], inplace = True)
df.sort_values(['ticker','date'], inplace = True)

In [18]:
#to normalize the data
#df['cls_norm'] = df.groupby('ticker')['adj_close'].transform(lambda x: (x - x.mean()) / x.std())
df['ret'] = df.groupby("ticker")['Adj Close'].pct_change()
df['ret_norm'] = df.groupby('ticker')['ret'].transform(lambda x: (x - x.mean()) / x.std())

# Feature Creation

In [19]:
# to add moving averages to the dataframe
ma5 = df.groupby("ticker", as_index=False)['Adj Close'].rolling(window = 5).mean()
ma5 = pd.DataFrame(ma5)
ma5.rename(columns={ma5.columns[0]: "MA5"}, inplace = True)
ma5.reset_index(inplace = True)

ma10 = df.groupby("ticker", as_index=False)['Adj Close'].rolling(window = 10).mean()
ma10 = pd.DataFrame(ma10)
ma10.rename(columns={ma10.columns[0]: "MA10"}, inplace = True)
ma10.reset_index(inplace = True)

df_ma = df.merge(ma5[["MA5",'ticker','date']], on = ['ticker','date']).\
           merge(ma10[["MA10",'ticker','date']], on = ['ticker','date'])

In [21]:
#to write the function to find the Relative Strength Index
def RSI(stock, column="Adj Close", period=14):
    close = stock[column]
    delta = close.diff() 
    up, down = delta.copy(), delta.copy()

    up[up < 0] = 0
    down[down > 0] = 0
    
    roll_up = up.ewm(com=period - 1, adjust=False).mean()
    roll_down = down.ewm(com=period - 1, adjust=False).mean().abs()
    rs = roll_up / roll_down  
    rsi = 100-(100/(1+rs))
    
    stock['RSI'] = rsi
    return stock

In [22]:
df_ma_RSI  = df_ma.groupby("ticker").apply(RSI)

In [24]:
#to import CBOE VIX data
from pandas_datareader import data
VIX = data.DataReader('^VIX', 
                     start='2000-1-1', 
                     end='2020-1-1',
                     data_source='yahoo')
VIX = pd.DataFrame(VIX["Adj Close"])
VIX.rename(columns={VIX.columns[0]: "VIX"}, inplace = True)

In [498]:
VIX['VIX'] = VIX['VIX'].transform(lambda x: (x - x.mean()) / x.std())

In [26]:
df_ma_RSI_VIX = df_ma_RSI.merge(VIX, how="left", left_on="date", right_on="Date")

In [28]:
# to write a function that add the new highs and new lows within 3 trading days
def new_highs_lows(dataframe):
    dataframe['new_high'] = np.where(dataframe['Adj Close'] >= dataframe['Adj Close'].rolling(window=3).max(), 1, 0)
    dataframe['new_low'] = np.where(dataframe['Adj Close'] <= dataframe['Adj Close'].rolling(window=3).min(), 1, 0)
    return dataframe

In [111]:
df_ma_RSI_VIX_hl = df_ma_RSI_VIX.groupby('ticker').apply(new_highs_lows)

In [32]:
# to add the bollinger band and compare the stock prices' position relative to the band
def bollinger_band(dataframe):
    dataframe['ma20'] = dataframe['Adj Close'].rolling(window=20).mean()
    dataframe['std20'] = dataframe['Adj Close'].rolling(window=20).std()
    dataframe['upper_band'] = dataframe['ma20']+(dataframe['std20']*2)
    dataframe['lower_band'] = dataframe['ma20']-(dataframe['std20']*2)
    
    dataframe['in_upper_boll'] = np.where((dataframe['Adj Close'] > dataframe.ma20) & \
                                              (dataframe['Adj Close'] < dataframe.upper_band), 1, 0)
    dataframe['in_lower_boll'] = np.where((dataframe['Adj Close'] <= dataframe.ma20) & \
                                              (dataframe['Adj Close'] > dataframe.lower_band), 1, 0)
    
    dataframe['above_upper_boll'] = np.where((dataframe['Adj Close'] > dataframe.upper_band), 1, 0)
    dataframe['below_lower_boll'] = np.where((dataframe['Adj Close']< dataframe.lower_band), 1, 0)
    
    return dataframe

In [112]:
df_ma_RSI_VIX_boll = df_ma_RSI_VIX_hl.groupby('ticker').apply(bollinger_band)

In [113]:
df_ma_RSI_VIX_boll['RSI_overbought'] = np.where((df_ma_RSI_VIX_boll.RSI > 70), 1, 0)
df_ma_RSI_VIX_boll['RSI_oversold'] = np.where((df_ma_RSI_VIX_boll.RSI < 30), 1, 0)

In [497]:
df_ma_RSI_VIX_boll.set_index(['ticker','date'], inplace = True)

In [115]:
# to measures the consistency between direction of price movement and the direction of volume
df_ma_RSI_VIX_boll['vol_change'] = df_ma_RSI_VIX_boll.groupby("ticker")['Volume'].pct_change()
df_ma_RSI_VIX_boll['prc_vl_confirm_up'] = \
    np.where(((df_ma_RSI_VIX_boll.ret > 0) & (df_ma_RSI_VIX_boll.vol_change > 0)), 1, 0)

df_ma_RSI_VIX_boll['prc_vl_confirm_down'] = \
    np.where(((df_ma_RSI_VIX_boll.ret < 0) & (df_ma_RSI_VIX_boll.vol_change > 0)), 1, 0)

In [117]:
# to add a dummy variable on if the close price is above the 5 day moving average
# to add a dummy variable on if 5 day moving average is above 30 day moving average
df_ma_RSI_VIX_boll['cls_above_ma5'] = np.where(df_ma_RSI_VIX_boll['Adj Close'] > df_ma_RSI_VIX_boll.MA5, 1, -1)
df_ma_RSI_VIX_boll['ma5_above_ma10'] = np.where(df_ma_RSI_VIX_boll.MA5 > df_ma_RSI_VIX_boll.MA10, 1, -1)

In [118]:
#to set and shift tomorrow's stock price direction
df_ma_RSI_VIX_boll['tomorrow'] = np.where(df_ma_RSI_VIX_boll['ret'] > 0, "Rise", "Fall")
df_ma_RSI_VIX_boll['tomorrow'] = df_ma_RSI_VIX_boll.groupby("ticker")['tomorrow'].shift(-1)

In [120]:
df_ma_RSI_VIX_boll['cls_norm'] = df_ma_RSI_VIX_boll.groupby('ticker')['Adj Close'].transform(lambda x: (x - x.mean()) / x.std())
df_ma_RSI_VIX_boll['MA5'] = df_ma_RSI_VIX_boll.groupby('ticker')['MA5'].transform(lambda x: (x - x.mean()) / x.std())
df_ma_RSI_VIX_boll['MA10'] = df_ma_RSI_VIX_boll.groupby('ticker')['MA10'].transform(lambda x: (x - x.mean()) / x.std())
df_ma_RSI_VIX_boll['RSI'] = df_ma_RSI_VIX_boll.groupby('ticker')['RSI'].transform(lambda x: (x - x.mean()) / x.std())
df_ma_RSI_VIX_boll['vol_change'] = df_ma_RSI_VIX_boll.groupby('ticker')['vol_change'].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
# to finalize the columns to be used in machine learning training and testing
complete_df = df_ma_RSI_VIX_boll\
[['tomorrow', #tomorrow's pricedirection
'ret_norm','cls_norm',  #today's return normalized
'MA5', 'MA10',  # moving averages of 5 30 trading days
'RSI', 'RSI_overbought', 'RSI_oversold', #relative strength index, dummies on overbought and oversold 
'vol_change',# change of trading volume
'new_high', 'new_low', # new highs and new lows
'prc_vl_confirm_up','prc_vl_confirm_down',#dummies on if trading volume confirms the price change
'in_upper_boll', 'in_lower_boll', 'above_upper_boll', 'below_lower_boll', #dummies of stock price in bollinger
'cls_above_ma5', 'ma5_above_ma10', #dummies of stock closed above MA5; whether MA5 is above MA30
'VIX']]   #overall market volatility

complete_df.dropna(how="any", inplace = True, axis = 0)

# To use Decision Tree Classifier

In [421]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score

In [ ]:
#to split the data into X, y, and train and test dataframes based on 60% of the total dates
split_date = complete_df.index.levels[1].unique()[:int(complete_df.index.levels[1].nunique()*0.6)][-1]

complete_df.reset_index(inplace = True)

train = complete_df[complete_df.date <= split_date]
test = complete_df[complete_df.date > split_date]

train.set_index(['ticker','date'], inplace = True)
test.set_index(['ticker','date'], inplace = True)

X_train = train.drop(columns = ['tomorrow'])
y_train = train.tomorrow

X_test = test.drop(columns = ['tomorrow'])
y_test = test.tomorrow

#X_train, X_test, y_train, y_test = train_test_split(X_Cols, y_Cols, train_size = 0.6, shuffle = True, random_state = 1)

In [522]:
X = complete_df.drop(columns = ['tomorrow'])
y = complete_df[['date','ticker','tomorrow']]
X.set_index(['date','ticker'], inplace = True)
y.set_index(['date','ticker'], inplace = True)

In [323]:
# the to test the best max_depth, and min_samples_leaft value
# I kept changing the range to check the printed metrics with x from initially 0 to 20, and y initially from 0 to 25
# 7 for max-depth and 18 for min-samples-leaf seem to be the best parameters based on the
# metrics such as accuracy, precision, f1, roc, and loss

for x in range(5,20):
    for y in range(2,20):
        decision_tree_classifier = DecisionTreeClassifier(criterion='gini', max_depth=x, min_samples_leaf=y)
        decision_tree_classifier = decision_tree_classifier.fit(X_train, y_train)
        
        y_pred = decision_tree_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        macro_precision, macro_recall, macro_f1, _ = score(y_test,y_pred, average='macro')
        rise_precision, rise_recall, rise_f1 = score(y_test,y_pred)[0][1], score(y_test,y_pred)[1][1], \
                                               score(y_test,y_pred)[2][1]
        numerical_y_test = np.where(y_test=='Rise', 1, 0)
        numerical_y_pred = np.where(y_pred=='Rise',1,0)
        roc= roc_auc_score(numerical_y_test, numerical_y_pred)
        loss = log_loss(numerical_y_test,numerical_y_pred)
        
        #print(x,y)
        #print(accuracy)
        #print(macro_precision, macro_recall, macro_f1)
        #print(rise_precision, rise_recall, rise_f1)
        #print(roc)
        #print(loss)
        #print('-------------')

In [506]:
# to finalize the parameters and fit the model
decision_tree_classifier = DecisionTreeClassifier(criterion='gini', max_depth=7, min_samples_leaf=19)
decision_tree_classifier.fit(X_train, y_train)

In [525]:
# to double check and conduct a cross validation test using the parameters above to check the robustness of the model
cross_val_score(decision_tree_classifier, X, y, cv = 10)

# To use RandomForestClassifer

In [ ]:
# to use the for loop to tuning the best parameters
for x in range(30,70):
   # for z in ['auto', 'sqrt']:
   #     for a in [6, 8, 15]:
    #        for b in [20]:
                rand_frst_clf = RandomForestClassifier(n_estimators=x,
                                                       criterion= 'gini', 
                                                       min_samples_leaf = a,
                                                       min_samples_split = b,
                                                       random_state = 1)
                    rand_frst_clf.fit(X_train, y_train)
                    y_pred = rand_frst_clf.predict(X_test)

                    accuracy = accuracy_score(y_test, y_pred)
                    macro_precision, macro_recall, macro_f1, _ = score(y_test, y_pred, average='macro')
                    rise_precision, rise_recall, rise_f1 = score(y_test,y_pred)[0][1], score(y_test,y_pred)[1][1], \
                                                               score(y_test,y_pred)[2][1]
                    numerical_y_test = np.where(y_test=='Rise', 1, 0)
                    numerical_y_pred = np.where(y_pred=='Rise',1,0)
                    roc= roc_auc_score(numerical_y_test, numerical_y_pred)
                    loss = log_loss(numerical_y_test,numerical_y_pred)

                    #print(x, y, z)
                    #print(a, b)
                    #print('-----')
                    #print(accuracy)
                    #print(macro_precision, macro_recall, macro_f1)
                    #print(rise_precision, rise_recall, rise_f1)
                    #print(roc)
                    #print(loss)
                    #print('-------------')

In [552]:
# to finalize the parameters and fit the model
rand_frst_clf = RandomForestClassifier(n_estimators=47,
                                       criterion= 'gini', 
                                       min_samples_leaf = 8)
rand_frst_clf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=8, n_estimators=47)

In [534]:
# to double check the model using a cross validation test using the same parameters
cross_val_score(rand_frst_clf, X, y.values.ravel(), cv = 10)

# To Test on the Universe of Tickers

In [140]:
#to import the tickers and combine the ticker names from NYSE and NASDAQ
tickers_nyse = pd.read_csv('tickers_nyse.csv')
tickers_nyse = tickers_nyse['Symbol'].values.ravel()

tickers_nasd = pd.read_csv('tickers_nasd.csv')
tickers_nasd = tickers_nasd['Symbol'].values.ravel()
tickers_universe = np.concatenate([tickers_nyse,tickers_nasd])

In [141]:
output = quandl.get(str('EOD/'+tickers_universe[0]),\
                    start_date = split_date, end_date = '2019-01-10')[['Adj_Close', 'Volume']]

output = pd.DataFrame (output)
output['ticker'] = tickers_universe[0]

for ticker in tickers_universe[1:]:
    try:
        data = quandl.get(str('EOD/'+ticker), \
                          start_date = split_date, end_date = '2019-01-10')[['Adj_Close', 'Volume']]
        data = pd.DataFrame(data)
        data['ticker'] = ticker
        
        output = pd.concat([output, data], axis=0)
                
    except Exception:
        pass

output.reset_index(inplace= True)
output.rename(columns={output.columns[0]: "date"}, inplace = True)

In [142]:
stock_universe = output.copy()

In [303]:
saved = output.copy()

In [316]:
# to add basic features to the raw dataset

stock_universe.rename(columns={stock_universe.columns[1]: "Adj Close"}, inplace = True)
#stock_universe.reset_index(inplace = True)

stock_universe['ret'] = stock_universe.groupby("ticker")['Adj Close'].pct_change()
stock_universe['ret_norm'] = stock_universe.groupby('ticker')['ret'].transform(lambda x: (x - x.mean()) / x.std())
stock_universe.set_index(['ticker','date'], inplace = True)

In [318]:
#to write a function that add more feature to the raw dataset

def add_features (stock_universe):

    ma5 = stock_universe.groupby("ticker", as_index=False)['Adj Close'].rolling(window = 5).mean()
    ma5 = pd.DataFrame(ma5)
    ma5.rename(columns={ma5.columns[0]: "MA5"}, inplace = True)
    ma5.reset_index(inplace = True)

    ma10 = stock_universe.groupby("ticker", as_index=False)['Adj Close'].rolling(window = 10).mean()
    ma10 = pd.DataFrame(ma10)
    ma10.rename(columns={ma10.columns[0]: "MA10"}, inplace = True)
    ma10.reset_index(inplace = True)

    df_ma = stock_universe.merge(ma5[["MA5",'ticker','date']], on = ['ticker','date']).\
               merge(ma10[["MA10",'ticker','date']], on = ['ticker','date'])

    df_ma_RSI  = df_ma.groupby("ticker").apply(RSI)

    df_ma_RSI_VIX = df_ma_RSI.merge(VIX, how="left", left_on="date", right_on="Date")

    df_ma_RSI_VIX_hl = df_ma_RSI_VIX.groupby('ticker').apply(new_highs_lows)

    df_ma_RSI_VIX_boll = df_ma_RSI_VIX_hl.groupby('ticker').apply(bollinger_band)

    df_ma_RSI_VIX_boll['RSI_overbought'] = np.where((df_ma_RSI_VIX_boll.RSI > 70), 1, 0)
    df_ma_RSI_VIX_boll['RSI_oversold'] = np.where((df_ma_RSI_VIX_boll.RSI < 30), 1, 0)


    df_ma_RSI_VIX_boll['vol_change'] = df_ma_RSI_VIX_boll.groupby("ticker")['Volume'].pct_change()
    df_ma_RSI_VIX_boll['prc_vl_confirm_up'] = \
        np.where(((df_ma_RSI_VIX_boll.ret > 0) & (df_ma_RSI_VIX_boll.vol_change > 0)), 1, 0)

    df_ma_RSI_VIX_boll['prc_vl_confirm_down'] = \
        np.where(((df_ma_RSI_VIX_boll.ret < 0) & (df_ma_RSI_VIX_boll.vol_change > 0)), 1, 0)

    df_ma_RSI_VIX_boll['cls_above_ma5'] = np.where(df_ma_RSI_VIX_boll['Adj Close'] > df_ma_RSI_VIX_boll.MA5, 1, -1)
    df_ma_RSI_VIX_boll['ma5_above_ma10'] = np.where(df_ma_RSI_VIX_boll.MA5 > df_ma_RSI_VIX_boll.MA10, 1, -1)

    df_ma_RSI_VIX_boll['tomorrow'] = np.where(df_ma_RSI_VIX_boll['ret'] > 0, "Rise", "Fall")
    df_ma_RSI_VIX_boll['tomorrow'] = df_ma_RSI_VIX_boll.groupby("ticker")['tomorrow'].shift(-1)

    df_ma_RSI_VIX_boll['cls_norm'] = df_ma_RSI_VIX_boll.groupby('ticker')['Adj Close'].transform(lambda x: (x - x.mean()) / x.std())
    df_ma_RSI_VIX_boll['MA5'] = df_ma_RSI_VIX_boll.groupby('ticker')['MA5'].transform(lambda x: (x - x.mean()) / x.std())
    df_ma_RSI_VIX_boll['MA10'] = df_ma_RSI_VIX_boll.groupby('ticker')['MA10'].transform(lambda x: (x - x.mean()) / x.std())
    df_ma_RSI_VIX_boll['RSI'] = df_ma_RSI_VIX_boll.groupby('ticker')['RSI'].transform(lambda x: (x - x.mean()) / x.std())
    df_ma_RSI_VIX_boll['vol_change'] = df_ma_RSI_VIX_boll.groupby('ticker')['vol_change'].transform(lambda x: (x - x.mean()) / x.std())

    complete_df = df_ma_RSI_VIX_boll\
    [['ticker','date','tomorrow', #tomorrow's stock price direction
    'ret_norm','cls_norm',  #today's return normalized
    'MA5', 'MA10',  # moving averages of 5 30 trading days
    'RSI', 'RSI_overbought', 'RSI_oversold', #relative strength index, dummies on overbought and oversold 
    'vol_change',# change of trading volume
    'new_high', 'new_low', # new highs and new lows
    'prc_vl_confirm_up','prc_vl_confirm_down',#dummies on if trading volume confirms the price change
    'in_upper_boll', 'in_lower_boll', 'above_upper_boll', 'below_lower_boll', #dummies of stock price in bollinger
    'cls_above_ma5', 'ma5_above_ma10', #dummies of stock closed above MA5; whether MA5 is above MA30
    'VIX']] 
    complete_df.dropna(inplace=True)

    return complete_df

In [ ]:
# to use the function above to add features to the universe of stock dataset

stock_universe_features = add_features(stock_universe)

In [493]:
# to get all the ticker names
unique_tics = stock_universe_features.ticker.unique()

In [482]:
# to get the metrics for all stocks in the stock universe
accuracy_ranking = pd.DataFrame({"DT_Accuracy": range(0)})

for stocks in unique_tics:
    
    test = stock_universe_features[stock_universe_features.ticker==stocks]
    test.set_index(['ticker','date'], inplace = True)
    
    X_test = test.drop(columns = ['tomorrow'])
    y_test = test.tomorrow
    
    y_pred = decision_tree_classifier.predict(X_test) # using the decision tree classifier fitted above
    
    #accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy = pd.Series(accuracy).to_frame(0)
    accuracy.rename(columns={accuracy.columns[0]: 'DT_Accuracy'},
                  index={accuracy.index[0]: stocks}, inplace=True)
    #macro metrics
    macros = pd.Series(score(y_test,y_pred, average='macro')).to_frame(0).T.iloc[:,0:3]
    macros.rename(columns={macros.columns[0]: 'DT_Macro Precision',
                          macros.columns[1]: 'DT_Macro Recall',
                          macros.columns[2]: 'DT_Macro F1'},
                  index={macros.index[0]: stocks}, inplace=True)
    #metrics for rises
    rise_precision, rise_recall, rise_f1 = score(y_test,y_pred)[0][1], score(y_test,y_pred)[1][1], \
                                               score(y_test,y_pred)[2][1]
    rise_metrics = pd.DataFrame({'DT_Rise Precision':[rise_precision], 'DT_Rise Recall': [rise_recall], \
                                 'DT_Rise F1': [rise_f1]})
    rise_metrics.rename(index={rise_metrics.index[0]: stocks}, inplace=True)
    #ROC, log loss
    numerical_y_test = np.where(y_test=='Rise', 1, 0)
    numerical_y_pred = np.where(y_pred=='Rise', 1, 0)
    
    roc= roc_auc_score(numerical_y_test, numerical_y_pred)
    loss = log_loss(numerical_y_test,numerical_y_pred)
    roc_loss = pd.DataFrame({'DT_ROC':[roc], 'DT_Loss': [loss]})
    roc_loss.rename(index={roc_loss.index[0]: stocks}, inplace=True)
    
    dt_metrics = accuracy.merge(macros, left_index=True, right_index=True).\
                          merge(rise_metrics, left_index=True, right_index=True).\
                          merge(roc_loss, left_index=True, right_index=True)
    
    ### to repete the above steps using Random Forest Classifier
    y_pred = rand_frst_clf.predict(X_test) # using the random forest fitted above
    #accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy = pd.Series(accuracy).to_frame(0)
    accuracy.rename(columns={accuracy.columns[0]: 'RF_Accuracy'},
                  index={accuracy.index[0]: stocks}, inplace=True)
    #macro metrics
    macros = pd.Series(score(y_test,y_pred, average='macro')).to_frame(0).T.iloc[:,0:3]
    macros.rename(columns={macros.columns[0]: 'RF_Macro Precision',
                          macros.columns[1]: 'RF_Macro Recall',
                          macros.columns[2]: 'RF_Macro F1'},
                  index={macros.index[0]: stocks}, inplace=True)
    #metrics for rises
    rise_precision, rise_recall, rise_f1 = score(y_test,y_pred)[0][1], score(y_test,y_pred)[1][1], \
                                               score(y_test,y_pred)[2][1]
    rise_metrics = pd.DataFrame({'RF_Rise Precision':[rise_precision], 'RF_Rise Recall': [rise_recall], \
                                 'RF_Rise F1': [rise_f1]})
    rise_metrics.rename(index={rise_metrics.index[0]: stocks}, inplace=True)
    #ROC, log loss
    numerical_y_test = np.where(y_test=='Rise', 1, 0)
    numerical_y_pred = np.where(y_pred=='Rise', 1, 0)
    
    roc= roc_auc_score(numerical_y_test, numerical_y_pred)
    loss = log_loss(numerical_y_test,numerical_y_pred)
    roc_loss = pd.DataFrame({'RF_ROC':[roc], 'RF_Loss': [loss]})
    roc_loss.rename(index={roc_loss.index[0]: stocks}, inplace=True)
    
    rf_metrics = accuracy.merge(macros, left_index=True, right_index=True).\
                          merge(rise_metrics, left_index=True, right_index=True).\
                          merge(roc_loss, left_index=True, right_index=True)
    
    #to merge decision tree metrics and random forest metrics
    all_metrics = dt_metrics.merge(rf_metrics, how="outer",left_index=True, right_index=True)
    
    #to concatenate metrics for different tickers
    accuracy_ranking = pd.concat([accuracy_ranking, all_metrics], join="outer", axis = 0)

In [553]:
accuracy_ranking.sort_values('DT_Accuracy', ascending=False).head(20)
#accuracy_ranking.sort_values('RF_Accuracy', ascending=False).head(20)
#accuracy_ranking.sort_values('DT_ROC').head(20)

,DT_Accuracy,DT_Macro Precision,DT_Macro Recall,DT_Macro F1,DT_Rise Precision,DT_Rise Recall,DT_Rise F1,DT_ROC,DT_Loss,RF_Accuracy,RF_Macro Precision,RF_Macro Recall,RF_Macro F1,RF_Rise Precision,RF_Rise Recall,RF_Rise F1,RF_ROC,RF_Loss
DELT,0.615385,0.633333,0.595238,0.575163,0.600000,0.857143,0.705882,0.595238,13.284391,0.615385,0.619048,0.619048,0.615385,0.666667,0.571429,0.615385,0.619048,13.284268
CATS,0.589074,0.595606,0.591375,0.585330,0.564706,0.699029,0.624729,0.591375,14.193105,0.498812,0.496330,0.496500,0.491642,0.484848,0.388350,0.431267,0.496500,17.310570
ADSW,0.579606,0.590315,0.576472,0.561364,0.562982,0.771127,0.650817,0.576472,14.520122,0.508050,0.510664,0.510026,0.501516,0.521327,0.387324,0.444444,0.510026,16.991492
FIXD,0.571429,0.575413,0.573642,0.569668,0.549815,0.653509,0.597194,0.573642,14.802541,0.511727,0.509855,0.509636,0.507780,0.497487,0.434211,0.463700,0.509636,16.864520
IBKCP,0.568698,0.576165,0.567705,0.555916,0.553763,0.733967,0.631256,0.567705,14.896891,0.507766,0.508774,0.508391,0.502636,0.513595,0.403800,0.452128,0.508391,17.001320
BOMN,0.566234,0.561375,0.554754,0.547667,0.575875,0.718447,0.639309,0.554754,14.981981,0.506494,0.517347,0.516136,0.501377,0.557143,0.378641,0.450867,0.516136,17.045239
SQ,0.565385,0.544580,0.535478,0.522803,0.589161,0.764172,0.665350,0.535478,15.011324,0.488462,0.496791,0.496789,0.488461,0.561765,0.433107,0.489117,0.496789,17.668065
MULE,0.564103,0.564891,0.562279,0.558774,0.560976,0.661871,0.607261,0.562279,15.055575,0.494505,0.496778,0.497020,0.486058,0.505051,0.359712,0.420168,0.497020,17.459305
FNKO,0.562500,0.542056,0.532628,0.517447,0.584112,0.771605,0.664894,0.532628,15.110962,0.513889,0.529221,0.528219,0.512737,0.598214,0.413580,0.489051,0.528219,16.789808
AAAP,0.561151,0.551196,0.534539,0.505850,0.568493,0.819079,0.671159,0.534539,15.157577,0.455036,0.460561,0.460918,0.454823,0.502075,0.398026,0.444037,0.460918,18.822563


In [551]:
#accuracy_ranking.describe()

,DT_Accuracy,DT_Macro Precision,DT_Macro Recall,DT_Macro F1,DT_Rise Precision,DT_Rise Recall,DT_Rise F1,DT_ROC,DT_Loss,RF_Accuracy,RF_Macro Precision,RF_Macro Recall,RF_Macro F1,RF_Rise Precision,RF_Rise Recall,RF_Rise F1,RF_ROC,RF_Loss
count,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000,4334.000000
mean,0.503351,0.506509,0.504833,0.468768,0.500455,0.758349,0.602056,0.504833,17.153962,0.503152,0.503523,0.503492,0.500838,0.501200,0.445823,0.470677,0.503492,17.160705
std,0.022335,0.021324,0.016306,0.021817,0.032512,0.039035,0.028259,0.016306,0.771443,0.015967,0.015785,0.015540,0.015860,0.034690,0.030041,0.022190,0.015540,0.551467
min,0.305750,0.251372,0.302137,0.265561,0.265306,0.373626,0.382022,0.302137,13.284391,0.269231,0.256250,0.269231,0.259370,0.210526,0.142857,0.210526,0.269231,13.284268
25%,0.490211,0.494217,0.495728,0.455941,0.482588,0.742482,0.587558,0.495728,16.626626,0.493704,0.494402,0.494491,0.491604,0.482112,0.429080,0.458101,0.494491,16.852772
50%,0.505247,0.506633,0.504827,0.468647,0.505897,0.763589,0.606545,0.504827,17.088491,0.502630,0.503503,0.503450,0.500808,0.506117,0.445495,0.472312,0.503450,17.178712
75%,0.518619,0.519547,0.514254,0.481900,0.523244,0.781242,0.622236,0.514254,17.607784,0.512067,0.512375,0.512228,0.509755,0.524315,0.462531,0.484882,0.512228,17.487017
max,0.615385,0.633333,0.595238,0.585330,0.600000,0.943864,0.705882,0.595238,23.978911,0.615385,0.619048,0.619048,0.615385,0.666667,0.636364,0.615385,0.619048,25.240213


In [2]:
# two export the two datasets to CSV files
accuracy_ranking.to_csv('metrics for stock universe.csv')
accuracy_ranking_sorted.to_csv('top 20 accuracy from universe.csv')